In [ ]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

In [ ]:
!curl https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

In [ ]:
%pwd
%ls -lh

In [ ]:
%pip install -U -r requirements.txt --quiet

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os
os.environ["DATASET"] = "realfill_data_release_full"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "RealBench" # Replace this with "Qualitative"
os.environ["DATASET_NUMBER"] = "23"
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}{os.environ["DATASET_NUMBER"]}'
os.environ["OUTPUT_DIR"] = f'{os.environ["BENCHMARK"]}{os.environ["DATASET_NUMBER"]}-model'
os.environ["VAL_IMG"] = f'{os.environ["TRAIN_DIR"]}/target/target.png'
os.environ["VAL_MASK"] = f'{os.environ["TRAIN_DIR"]}/target/mask.png'
os.environ["OUTPUT_IMG_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'

In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100

# Unused:
# --checkpointing_steps 100 \

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r $OUTPUT_DIR.zip $OUTPUT_DIR/logs